### Real time evolution with TEBD

In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb as qu
import quimb.tensor as qtn
import numpy as np

/home/sijingdu/anaconda3/envs/BPSU/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [2]:
L = 44
zeros = '0' * ((L - 2) // 3)
binary = zeros + '1' + zeros + '1' + zeros
print('psi0:', f"|{binary}>")

psi0: |00000000000000100000000000000100000000000000>


In [3]:
psi0 = qtn.MPS_computational_state(binary)
psi0.show()  # prints ascii representation of state

     1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1    
... >─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─>─ ...
    │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │     
                                 ...                                  
     1 1 1 1 1 1 1 1 
    >─>─>─>─>─>─>─>─<
    │ │ │ │ │ │ │ │ │


In [8]:
builder = qtn.SpinHam1D(S=1/2)

# specify the interaction term (defaults to all sites)
builder += 0.5, '+', '-'
builder += 0.5, '-', '+'
builder += 1.0, 'Z', 'Z'

# add random z-fields to each site
np.random.seed(2)
for i in range(L):
    builder[i] += 2 * np.random.rand() - 1, 'Z'
    
H = builder.build_local_ham(L)

tebd = qtn.TEBD(psi0, H)

# Since entanglement will not grow too much, we can set quite
#     a small cutoff for splitting after each gate application
tebd.split_opts['cutoff'] = 1e-12
# times we are interested in
ts = np.linspace(0, 80, 101)

mz_t_j = []  # z-magnetization
be_t_b = []  # block entropy
sg_t_b = []  # schmidt gap

# range of bonds, and sites
js = np.arange(0, L)
bs = np.arange(1, L)

In [9]:
# generate the state at each time in ts
#     and target error 1e-3 for whole evolution
for psit in tebd.at_times(ts, tol=1e-3):
    mz_j = []
    be_b = []
    sg_b = []
    
    # there is one more site than bond, so start with mag
    #     this also sets the orthog center to 0
    mz_j += [psit.magnetization(0)]
    
    for j in range(1, L):
        # after which we only need to move it from previous site
        mz_j += [psit.magnetization(j, cur_orthog=j - 1)]
        be_b += [psit.entropy(j, cur_orthog=j)]
        sg_b += [psit.schmidt_gap(j, cur_orthog=j)]
        
    mz_t_j += [mz_j]
    be_t_b += [be_b]
    sg_t_b += [sg_b]

t=80, max-bond=22: 100%|##########| 101/101 [01:48<00:00,  1.07s/it]  


In [10]:
import matplotlib.pyplot as plt
with plt.style.context(qu.NEUTRAL_STYLE):
    plt.figure(figsize=(12, 7))

    # plot the magnetization
    ax1 = plt.subplot(131)
    plt.pcolormesh(js, ts, np.real(mz_t_j), vmin=-0.5, vmax=0.5)
    plt.set_cmap('RdYlBu')
    plt.colorbar()
    plt.title('Z-Magnetization')
    plt.xlabel('Site')
    plt.ylabel('time [ $Jt$ ]')

    # plot the entropy
    ax2 = plt.subplot(132, sharey=ax1)
    plt.pcolormesh(bs, ts, be_t_b)
    plt.setp(ax2.get_yticklabels(), visible=False)
    plt.set_cmap('viridis'), plt.colorbar()
    plt.title('Block Entropy')
    plt.xlabel('Bond')

    # plot the schmidt gap
    ax3 = plt.subplot(133, sharey=ax1)
    plt.pcolormesh(bs, ts, sg_t_b, vmin=0, vmax=1)
    plt.setp(ax3.get_yticklabels(), visible=False)
    plt.set_cmap('magma_r')
    plt.colorbar()
    plt.title('Schmidt Gap')
    plt.xlabel('Bond')

    plt.show()

### Imaginary time evolution with TEBD

In [ ]:
import collections
import functools
import itertools
import os
import time
from math import pi, log2, log10

import xyzpy as xyz
import autoray as ar
import quimb as qu
import quimb.tensor as qtn
from quimb.tensor.circuit import Gate

from quimb.experimental.belief_propagation.l2bp import (
    L2BP,
    compress_l2bp,
    contract_l2bp,
)
from quimb.experimental.belief_propagation.l1bp import (
    contract_l1bp,
)